Quickstart: Build and test simple Agent using Mosaic AI Agent Framework 
- This notebook demonstrates how to build and test a generative AI agent using Mosaic AI Agent Framework on Databricks
- Automatically log traces from LLM calls for ease of debugging using Mlflow 
- The agent uses an LLM served on Databricks Foundation Model API 
- The agent has access to a single tool, the built-in Python code interpreter tool on Databricks Unity Catalog. It can use this tool to run LLM-generated code in order to respond to user questions. 

Databricks_openai SDK will be used to query the LLM endpoint 

In [0]:
%pip list

In [0]:
%pip install -U -qqqq mlflow databricks-openai databricks-agents

In [0]:
 dbutils.library.restartPython()

In [0]:
%pip show databricks-agents

In [0]:
import warnings
warnings.filterwarnings("ignore")

### Pick the first LLM API available in the databricks ws

In [0]:
# from a set of candidates, can override and simplify it to just specify LLM_ENDPOINT_NAME 
LLM_ENDPOINT_NAME = None

from databricks.sdk import WorkspaceClient 
def is_endpoint_available(endpoint_name): 
    try: 
        w = WorkspaceClient() 
        client = w.serving_endpoints.get_open_ai_client() 
        client.chat.completions.create(model=endpoint_name, messages=[{"role": "user", "content": "What is AI?"}])
        return True 
    except Exception as e: 
        #print(f"Error: {e}") 
        return False 

for candidate in ["databricks-claude-3-7-sonnet","databricks-meta-llama-3-3-70b-instruct","databricks-codellama-3-70b-instruct"]:
    if is_endpoint_available(candidate):
        LLM_ENDPOINT_NAME = candidate
        break
assert LLM_ENDPOINT_NAME is not None, "Please specify LLM_ENDPOINT_NAME"
print(f"LLM_ENDPOINT_NAME: {LLM_ENDPOINT_NAME}")

    

### Do a LLM Call

In [0]:
import json 
import mlflow 
from databricks.sdk import WorkspaceClient 

# Automatically log traces from LLM calls 
mlflow.openai.autolog() 

open_client = WorkspaceClient().serving_endpoints.get_open_ai_client() 
prompt = "What is the capital of Finland?" 

response = open_client.chat.completions.create(model=LLM_ENDPOINT_NAME, messages=[{"role": "user", "content": prompt}])


In [0]:
print(response)

In [0]:
print(type(response))

In [0]:
response.choices[0].message.content

### Wrap in function 

In [0]:
def run_llm(prompt): 
    """ 
    Send a user prompt to the LLM and return the responses.  
    """
    results = []
    response = open_client.chat.completions.create(
        model=LLM_ENDPOINT_NAME, messages=[{"role": "user", "content": prompt}]
    )

    msg = response.choices[0].message
    results.append(msg.to_dict())
    return results


In [0]:
print(type(response.choices[0].message))

In [0]:
resps = run_llm("What is the capital of Finland")
for mes in resps: 
    print(f"{mes['role']}: {mes['content']}")


### Add tool and create a Agent 


In [0]:
from databricks_openai import UCFunctionToolkit, DatabricksFunctionClient
from databricks.sdk import WorkspaceClient 

mlflow.openai.autolog() 

openai_client = WorkspaceClient().serving_endpoints.get_open_ai_client() 
databricks_client = DatabricksFunctionClient()

# Load Databricks built-in tools
builtin_tools = UCFunctionToolkit(
    function_names=["system.ai.python_exec"], client=databricks_client
).tools 




In [0]:
from databricks.sdk import WorkspaceClient

# Refresh Unity Catalog metadata for system.ai functions
# This ensures the python_exec tool is discoverable in your workspace
w = WorkspaceClient()
list(w.schemas.list("system"))
w.schemas.get("system.ai")
l = list(w.functions.list(catalog_name="system", schema_name="ai"))
print(l)

In [0]:
def call_tool(tool_name, parameters): 
    if tool_name == "system__ai__python_exec": 
        return DatabricksFunctionClient().execute_function("system.ai.python.exec", parameters=parameters)
    raise ValueError(f"Unknown tool: {tool_name}")

def run_agent(prompt): 
    result_msgs = []
    response = openai_client.chat.completions.create(
        model=LLM_ENDPOINT_NAME,
        messages=[{"role": "user", "content": prompt}],
        tools=builtin_tools,
    )

    msg = response.choices[0].message
    result_msgs.append(msg.to_dict())
    while msg.tool_calls:
        call = msg.tool_calls[0]   
        tool_result = call_tool(call.function.name, tool_call.function.arguments)
        result_msgs.append({
            "role": "tool", 
            "content": tool_result.value, 
            "name": call.function.name, 
            "tool_call_id": call.id
        })
    
    return result_msgs

In [0]:
answer = run_agent("What is the best place to visit in Vietnam")
for mes in answer: 
    print(f"{mes['role']}: {mes['content']}")